In [1]:
import pandas as pd
import numpy as np

from flatlib import const
from flatlib.chart import Chart
from flatlib.datetime import Datetime
from flatlib.geopos import GeoPos
from flatlib.object import GenericObject

from flatlib.tools import arabicparts
from flatlib.dignities import essential

from _astro_constants import AstrologicalConstants, AstrologicalPoints

pd.set_option('display.max_columns', 200)  # or 1000
pd.set_option('display.max_rows', 200)  # or 1000
pd.set_option('display.max_colwidth', 100)  # or 199

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
data = {'date':['2015/03/13', '2016/05/23', '2018/10/03'], 'utc_time':['14:00', '12:30', '00:15'], 'lon':['38n32', '08n07', '14s42'], 'lat':['8w54', '18e14','36w04']}
df = pd.DataFrame(data)
df

,date,utc_time,lon,lat
0,2015/03/13,14:00,38n32,8w54
1,2016/05/23,12:30,08n07,18e14
2,2018/10/03,00:15,14s42,36w04


In [4]:
df['charts'] = AstrologicalPoints.charts_calculate(df, 'date', 'utc_time', 'lon', 'lat')

In [5]:
df.charts

0    <flatlib.chart.Chart object at 0x7ff0b69a4520>
1    <flatlib.chart.Chart object at 0x7ff0b69a4550>
2    <flatlib.chart.Chart object at 0x7ff0b69bc3d0>
Name: charts, dtype: object

In [6]:
rulers_constants = AstrologicalConstants.rulers_constants
str(rulers_constants)

"['ASC', 'DESC', 'MC', 'IC', 'PARS_FORTUNA']"

In [7]:
necessary_constants = AstrologicalConstants.necessary_constants
str(necessary_constants)

"['SUN', 'MOON', 'SATURN', 'URANUS', 'NEPTUNE', 'PLUTO', 'CHIRON', 'NORTH_NODE', 'SOUTH_NODE']"

In [8]:
for obj in rulers_constants + necessary_constants:
    df[str.lower(obj)] = AstrologicalPoints.calculate_astro_objects(df.charts, obj)

In [9]:
rulers_col_names = AstrologicalPoints.rulers_col_names(rulers_constants)
rulers_col_names

['ruler_asc', 'ruler_desc', 'ruler_mc', 'ruler_ic', 'ruler_pars_fortuna']

In [10]:
for name, obj in zip(rulers_col_names, rulers_constants):
    obj = str.lower(obj)
    df[name] = AstrologicalPoints.ruler_of_object(df[obj])
    df[name] = AstrologicalPoints.chart_object_attributes(df, 'charts', name)

In [11]:
antes_objects = AstrologicalConstants.antes_objects
str(antes_objects)

"['pars_fortuna', 'uranus', 'neptune', 'pluto', 'chiron']"

In [12]:
for obj in antes_objects:
    antes_name = 'antes_' + obj
    df[antes_name] = AstrologicalPoints.antes_objects(df, obj)

In [13]:
df

,date,utc_time,lon,lat,charts,asc,desc,mc,ic,pars_fortuna,sun,moon,saturn,uranus,neptune,pluto,chiron,north_node,south_node,ruler_asc,ruler_desc,ruler_mc,ruler_ic,ruler_pars_fortuna,antes_pars_fortuna,antes_uranus,antes_neptune,antes_pluto,antes_chiron
0,2015/03/13,14:00,38n32,8w54,<flatlib.chart.Chart object at 0x7ff0b69a4520>,<Asc Cancer +27:24:38>,<Desc Capricorn +27:24:38>,<MC Aries +13:01:44>,<IC Libra +13:01:44>,<Pars Fortuna Aries +25:28:18>,<Sun Pisces +22:39:56 +00:59:52>,<Moon Sagittarius +20:43:35 +13:13:02>,<Saturn Sagittarius +04:55:45 +00:00:06>,<Uranus Aries +15:06:33 +00:03:14>,<Neptune Pisces +07:50:38 +00:02:13>,<Pluto Capricorn +15:14:35 +00:01:02>,<Chiron Pisces +17:36:15 +00:03:43>,<North Node Libra +11:08:52 -00:03:11>,<South Node Aries +11:08:52 -00:03:11>,<Moon Sagittarius +20:43:35 +13:13:02>,<Saturn Sagittarius +04:55:45 +00:00:06>,<Mars Aries +16:27:08 +00:45:19>,<Venus Aries +25:21:06 +01:12:41>,<Mars Aries +16:27:08 +00:45:19>,<Pars Fortuna Virgo +04:31:42>,<Uranus Virgo +14:53:27 +00:03:14>,<Neptune Libra +22:09:22 +00:02:13>,<Pluto Sagittarius +14:45:25 +00:01:02>,<Chiron Libra +12:23:45 +00:03:43>
1,2016/05/23,12:30,08n07,18e14,<flatlib.chart.Chart object at 0x7ff0b69a4550>,<Asc Virgo +27:12:45>,<Desc Pisces +27:12:45>,<MC Gemini +27:30:28>,<IC Sagittarius +27:30:28>,<Pars Fortuna Aries +15:36:29>,<Sun Gemini +02:47:58 +00:57:39>,<Moon Sagittarius +21:11:41 +12:19:41>,<Saturn Sagittarius +13:54:21 -00:04:19>,<Uranus Aries +22:47:40 +00:02:49>,<Neptune Pisces +11:54:55 +00:00:42>,<Pluto Capricorn +17:11:35 -00:00:57>,<Chiron Pisces +24:43:00 +00:01:47>,<North Node Virgo +18:00:29 -00:03:11>,<South Node Pisces +18:00:29 -00:03:11>,<Mercury Taurus +14:22:23 +00:04:22>,<Jupiter Virgo +13:33:03 +00:02:30>,<Mercury Taurus +14:22:23 +00:04:22>,<Jupiter Virgo +13:33:03 +00:02:30>,<Mars Sagittarius +01:25:22 -00:21:00>,<Pars Fortuna Virgo +14:23:31>,<Uranus Virgo +07:12:20 +00:02:49>,<Neptune Libra +18:05:05 +00:00:42>,<Pluto Sagittarius +12:48:25 -00:00:57>,<Chiron Libra +05:17:00 +00:01:47>
2,2018/10/03,00:15,14s42,36w04,<flatlib.chart.Chart object at 0x7ff0b69bc3d0>,<Asc Gemini +05:25:08>,<Desc Sagittarius +05:25:08>,<MC Pisces +07:39:39>,<IC Virgo +07:39:39>,<Pars Fortuna Pisces +13:18:26>,<Sun Libra +09:44:33 +00:59:03>,<Moon Cancer +17:37:52 +14:05:52>,<Saturn Capricorn +03:06:46 +00:02:33>,<Uranus Taurus +01:23:25 -00:02:14>,<Neptune Pisces +14:24:42 -00:01:27>,<Pluto Capricorn +18:45:26 +00:00:03>,<Chiron Pisces +29:41:06 -00:02:40>,<North Node Leo +02:19:59 -00:03:11>,<South Node Aquarius +02:19:59 -00:03:11>,<Mercury Libra +18:46:13 +01:38:51>,<Jupiter Scorpio +22:27:07 +00:11:21>,<Jupiter Scorpio +22:27:07 +00:11:21>,<Mercury Libra +18:46:13 +01:38:51>,<Jupiter Scorpio +22:27:07 +00:11:21>,<Pars Fortuna Libra +16:41:34>,<Uranus Leo +28:36:35 -00:02:14>,<Neptune Libra +15:35:18 -00:01:27>,<Pluto Sagittarius +11:14:34 +00:00:03>,<Chiron Libra +00:18:54 -00:02:40>


In [14]:
# Rulies for ASC
df.ruler_asc = np.where((df.ruler_asc == df.moon) | (df.ruler_desc == df.moon), None, df.ruler_asc)
# Rulies for DESC
df.ruler_desc = np.where((df.ruler_desc == df.moon) | (df.ruler_asc == df.moon), None, df.ruler_desc)

# Rulies for MC
df.ruler_mc = np.where((df.ruler_mc == df.ruler_asc) | (df.ruler_mc == df.ruler_desc) | (df.ruler_mc == df.saturn) | (df.ruler_mc == df.ruler_pars_fortuna) 
                       | (df.ruler_mc == df.moon), None, df.ruler_mc)
# Rulies for IC
df.ruler_ic = np.where((df.ruler_ic == df.ruler_desc) | (df.ruler_ic == df.ruler_asc) | (df.ruler_ic == df.saturn) | (df.ruler_ic == df.ruler_pars_fortuna) 
                       | (df.ruler_ic == df.moon), None, df.ruler_ic)

# Rulies for PARS FORTUNA
df.ruler_pars_fortuna = np.where((df.ruler_pars_fortuna == df.ruler_asc) | (df.ruler_pars_fortuna == df.ruler_desc) | (df.ruler_pars_fortuna == df.saturn) 
                                 | (df.ruler_pars_fortuna == df.moon), None, df.ruler_pars_fortuna)

In [15]:
antes_objects = []

for col in df.columns:
    if col.startswith('antes'):
        AstrologicalPoints.rename_antes_objects(df, col)
        antes_objects.append(col)      

In [16]:
df

,date,utc_time,lon,lat,charts,asc,desc,mc,ic,pars_fortuna,sun,moon,saturn,uranus,neptune,pluto,chiron,north_node,south_node,ruler_asc,ruler_desc,ruler_mc,ruler_ic,ruler_pars_fortuna,antes_pars_fortuna,antes_uranus,antes_neptune,antes_pluto,antes_chiron
0,2015/03/13,14:00,38n32,8w54,<flatlib.chart.Chart object at 0x7ff0b69a4520>,<Asc Cancer +27:24:38>,<Desc Capricorn +27:24:38>,<MC Aries +13:01:44>,<IC Libra +13:01:44>,<Pars Fortuna Aries +25:28:18>,<Sun Pisces +22:39:56 +00:59:52>,<Moon Sagittarius +20:43:35 +13:13:02>,<Saturn Sagittarius +04:55:45 +00:00:06>,<Uranus Aries +15:06:33 +00:03:14>,<Neptune Pisces +07:50:38 +00:02:13>,<Pluto Capricorn +15:14:35 +00:01:02>,<Chiron Pisces +17:36:15 +00:03:43>,<North Node Libra +11:08:52 -00:03:11>,<South Node Aries +11:08:52 -00:03:11>,None,<Saturn Sagittarius +04:55:45 +00:00:06>,None,<Venus Aries +25:21:06 +01:12:41>,<Mars Aries +16:27:08 +00:45:19>,<Antes Pars Fortuna Virgo +04:31:42>,<Antes Uranus Virgo +14:53:27 +00:03:14>,<Antes Neptune Libra +22:09:22 +00:02:13>,<Antes Pluto Sagittarius +14:45:25 +00:01:02>,<Antes Chiron Libra +12:23:45 +00:03:43>
1,2016/05/23,12:30,08n07,18e14,<flatlib.chart.Chart object at 0x7ff0b69a4550>,<Asc Virgo +27:12:45>,<Desc Pisces +27:12:45>,<MC Gemini +27:30:28>,<IC Sagittarius +27:30:28>,<Pars Fortuna Aries +15:36:29>,<Sun Gemini +02:47:58 +00:57:39>,<Moon Sagittarius +21:11:41 +12:19:41>,<Saturn Sagittarius +13:54:21 -00:04:19>,<Uranus Aries +22:47:40 +00:02:49>,<Neptune Pisces +11:54:55 +00:00:42>,<Pluto Capricorn +17:11:35 -00:00:57>,<Chiron Pisces +24:43:00 +00:01:47>,<North Node Virgo +18:00:29 -00:03:11>,<South Node Pisces +18:00:29 -00:03:11>,<Mercury Taurus +14:22:23 +00:04:22>,<Jupiter Virgo +13:33:03 +00:02:30>,None,None,<Mars Sagittarius +01:25:22 -00:21:00>,<Antes Pars Fortuna Virgo +14:23:31>,<Antes Uranus Virgo +07:12:20 +00:02:49>,<Antes Neptune Libra +18:05:05 +00:00:42>,<Antes Pluto Sagittarius +12:48:25 -00:00:57>,<Antes Chiron Libra +05:17:00 +00:01:47>
2,2018/10/03,00:15,14s42,36w04,<flatlib.chart.Chart object at 0x7ff0b69bc3d0>,<Asc Gemini +05:25:08>,<Desc Sagittarius +05:25:08>,<MC Pisces +07:39:39>,<IC Virgo +07:39:39>,<Pars Fortuna Pisces +13:18:26>,<Sun Libra +09:44:33 +00:59:03>,<Moon Cancer +17:37:52 +14:05:52>,<Saturn Capricorn +03:06:46 +00:02:33>,<Uranus Taurus +01:23:25 -00:02:14>,<Neptune Pisces +14:24:42 -00:01:27>,<Pluto Capricorn +18:45:26 +00:00:03>,<Chiron Pisces +29:41:06 -00:02:40>,<North Node Leo +02:19:59 -00:03:11>,<South Node Aquarius +02:19:59 -00:03:11>,<Mercury Libra +18:46:13 +01:38:51>,<Jupiter Scorpio +22:27:07 +00:11:21>,None,None,None,<Antes Pars Fortuna Libra +16:41:34>,<Antes Uranus Leo +28:36:35 -00:02:14>,<Antes Neptune Libra +15:35:18 -00:01:27>,<Antes Pluto Sagittarius +11:14:34 +00:00:03>,<Antes Chiron Libra +00:18:54 -00:02:40>


In [17]:
houses_objects = AstrologicalConstants.houses_objects
houses_objects

['asc', 'desc', 'mc', 'ic']

In [18]:
main_objects = AstrologicalConstants.main_objects
str(main_objects)

"['moon', 'ruler_asc', 'ruler_desc', 'pars_fortuna', 'ruler_pars_fortuna', 'ruler_mc', 'ruler_ic', 'sun', 'saturn', 'uranus', 'neptune', 'pluto', 'chiron', 'north_node', 'south_node']"

In [19]:
str(antes_objects)

"['antes_pars_fortuna', 'antes_uranus', 'antes_neptune', 'antes_pluto', 'antes_chiron']"

In [28]:
cols_for_id = houses_objects + main_objects + antes_objects
len(cols_for_id)

24

In [39]:
 def chart_unique_ids(df: pd.DataFrame, cols_for_id: list):
        
        unique_ids = df[cols_for_id].apply(lambda x: [i.id for i in x])
#             unique_ids = set(unique_ids)
#             unique_ids.append(unique_id)
        return unique_ids   


In [82]:
# tmp

In [67]:
# def objects_for_aspects(df: pd.DataFrame, cols_for_id: list)
#     indexes = []
#     all_objects = []
all_col_obj_id = []
col_obj_id = tuple

for index, row in df[cols_for_id].iterrows():
    for col, obj_id in row.iteritems():
        if obj_id is not None:
            col_obj_id = (col, obj_id.id)
    all_col_obj_id.append(col_obj_id)                
    print(all_col_obj_id)    

[('antes_chiron', 'Antes Chiron')]
[('antes_chiron', 'Antes Chiron'), ('antes_chiron', 'Antes Chiron')]
[('antes_chiron', 'Antes Chiron'), ('antes_chiron', 'Antes Chiron'), ('antes_chiron', 'Antes Chiron')]


In [62]:
all_col_obj_id

[('asc', 'Asc'),
 ('desc', 'Desc'),
 ('mc', 'MC'),
 ('ic', 'IC'),
 ('moon', 'Moon'),
 ('ruler_desc', 'Saturn'),
 ('pars_fortuna', 'Pars Fortuna'),
 ('ruler_pars_fortuna', 'Mars'),
 ('ruler_ic', 'Venus'),
 ('sun', 'Sun'),
 ('saturn', 'Saturn'),
 ('uranus', 'Uranus'),
 ('neptune', 'Neptune'),
 ('pluto', 'Pluto'),
 ('chiron', 'Chiron'),
 ('north_node', 'North Node'),
 ('south_node', 'South Node'),
 ('antes_pars_fortuna', 'Antes Pars Fortuna'),
 ('antes_uranus', 'Antes Uranus'),
 ('antes_neptune', 'Antes Neptune'),
 ('antes_pluto', 'Antes Pluto'),
 ('antes_chiron', 'Antes Chiron'),
 ('asc', 'Asc'),
 ('desc', 'Desc'),
 ('mc', 'MC'),
 ('ic', 'IC'),
 ('moon', 'Moon'),
 ('ruler_asc', 'Mercury'),
 ('ruler_desc', 'Jupiter'),
 ('pars_fortuna', 'Pars Fortuna'),
 ('ruler_pars_fortuna', 'Mars'),
 ('sun', 'Sun'),
 ('saturn', 'Saturn'),
 ('uranus', 'Uranus'),
 ('neptune', 'Neptune'),
 ('pluto', 'Pluto'),
 ('chiron', 'Chiron'),
 ('north_node', 'North Node'),
 ('south_node', 'South Node'),
 ('antes_pars

In [111]:
sun = chart.get(const.SUN)

In [44]:
# tmp = chart_unique_ids(df, cols_for_id)

In [20]:
for obj in houses_objects + main_objects + antes_objects:
    print(obj)

asc
desc
mc
ic
moon
ruler_asc
ruler_desc
pars_fortuna
ruler_pars_fortuna
ruler_mc
ruler_ic
sun
saturn
uranus
neptune
pluto
chiron
north_node
south_node
antes_pars_fortuna
antes_uranus
antes_neptune
antes_pluto
antes_chiron


In [173]:
tmp = AstrologicalPoints.chart_unique_ids(df, main_objects)

In [175]:
tmp[0]

0    Moon
1    Moon
2    Moon
Name: moon, dtype: object

In [196]:
unique_ids = df[main_objects].apply(lambda x: list(set([str.upper(i.id) for i in x])), axis=1)

In [197]:
len(unique_ids[2])

12

In [198]:
unique_ids[1]

['SUN',
 'URANUS',
 'NORTH NODE',
 'PARS FORTUNA',
 'MERCURY',
 'SATURN',
 'PLUTO',
 'NEPTUNE',
 'MOON',
 'MARS',
 'SOUTH NODE',
 'JUPITER',
 'CHIRON']

In [90]:
chart = df.charts.values[2]
chart